In [17]:
import json

In [18]:
!ls ../multihop_dense_retrieval/data/hotpot

dev_retrieval_b50_k50_sp.json  hotpot_test_fullwiki_v1.json
dev_retrieval_top100_sp.json   hotpot_train_firsthop.json
hotpot_dev_firsthop.json       hotpot_train_v1.1.json
hotpot_dev_fullwiki_v1.json    hotpot_train_with_neg_v0.json
hotpot_dev_with_neg_v0.json    train_retrieval_b100_k100_sp.json
hotpot_qas_val.json


In [19]:
SPLIT = "test"

In [20]:
fp_hotpot_original = '../multihop_dense_retrieval/data/hotpot/hotpot_test_fullwiki_v1.json'
    
with open(fp_hotpot_original, 'r') as handle:
    hotpot_orig = json.load(handle)

In [21]:
# Training data length
len(hotpot_orig)

7405

In [22]:
# Example training object
hotpot_orig[0].keys()

dict_keys(['_id', 'question', 'context'])

In [23]:
hotpot_orig[0]

{'_id': '5adf9ba1554299025d62a2db',
 'question': "What position on the Billboard Top 100 did Alison Moyet's late summer hit achieve?",
 'context': [['The Other Side of Love',
   ['"The Other Side of Love" is a song by the British synthpop band Yazoo, released in 1982 as their third single.',
    " The single peaked at #13 on the UK Singles Chart, making it the band's least successful single and the only one of their four singles to miss the top 10.",
    ' The track was written by band members Vince Clarke and Alison Moyet, and was originally not included on either of the band\'s albums (it was later added to a reissue of "Upstairs at Eric\'s").',
    " It featured Stiff Records' all-girl band Sylvia and the Sapphires on backing vocals following a chance meeting on the B.A. Robertson show."]],
  ['All Cried Out (Alison Moyet song)',
   ['"All Cried Out" is a song by English singer-songwriter Alison Moyet.',
    ' It was written by Moyet and producers Jolley & Swain for her debut studio

In [24]:
def extract_sentences(sup_idx_list, context_list, doc_title=None):
    return_obj = []
    sup = {}
    for s in sup_idx_list:
        if s[0] not in sup:
            sup[s[0]] = []
        sup[s[0]].append(s[1])
    for c in context_list:
        if c[0] not in sup:
            continue
        if doc_title is not None and c[0] != doc_title:
            continue
        for i in sup[c[0]]:
            try:
                return_obj.append(c[1][i])
            except:
                print("IndexError: ignoring")
                print(c[0])
                print(c[1])
                print(sup[c[0]])
    return return_obj

In [25]:
output = []

for i in range(len(hotpot_orig)):
    orig_obj = hotpot_orig[i]
    q_obj = {
        "id": orig_obj["_id"],
        "question": orig_obj["question"],
#         "level": "",
#         "type": "",
#         "answers": [],
#         "final_answer": "",
    }
    output.append(q_obj)

In [26]:
output[19]

{'id': '5abbc41955429931dba144fb',
 'question': "What political role was Nattrass critical of during Mbeki's faulty policy?"}

In [27]:
# Store results to disk

output_fp = f"../multihop_dense_retrieval/data/hotpot/hotpot_{SPLIT}_firsthop.json"
with open(output_fp, 'w') as fp:
    json.dump({"data": output}, fp, indent=4)